In [ ]:
créer un environnement : conda create (voir dans le cours)
conda activagte nomenvrionement
conda install ipykernel notebook
pip install pandas
pip install scrapy 

In [ ]:
import requests
import pandas as pd
from scrapy import Selector


url = "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
response = requests.get(url)

print(response.status_code)
print(response.text[:500])

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3 ',
    "Accept-Language": "fr-FR,fr;q=0.9"
}
response_imdb = requests.get(url, headers=headers)
sel_imdb = Selector(text=response_imdb.text)

titre = sel_imdb.css('div.product_main h1::text').get()
print("Titre : ", titre)

prix = sel_imdb.css('div.product_main p.price_color::text').get()
print("prix : ", prix)

stock_parts = sel_imdb.css('p.instock.availability::text').getall()
Stock_availability = ''.join(part.strip() for part in stock_parts).strip()
print("stock :", Stock_availability)



imageurl = sel_imdb.css('div.item.active img::attr(src)').get()
print("url image: ",imageurl)

category_url = "http://books.toscrape.com/catalogue/category/books/poetry_23/index.html"
response_cat = requests.get(category_url, headers=headers)
sel_cat = Selector(text=response_cat.text)
category = sel_cat.css('div.page-header.action h1::text, div.page-header h1::text').get()
if category:
    category = category.strip()
print("Category:", category)



200


<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    A Light in the Attic | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charse
Titre :  A Light in the Attic
prix :  Â£51.77
stock : In stock (22 available)
url image:  ../../media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg
Category: Poetry


In [10]:
import requests
import pandas as pd
from scrapy import Selector
from urllib.parse import urljoin

request_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

base_url = "https://books.toscrape.com/"

category_urls = {
    "Poetry": "https://books.toscrape.com/catalogue/category/books/poetry_23/index.html",
    "Mystery": "https://books.toscrape.com/catalogue/category/books/mystery_3/index.html",
    "Fiction": "https://books.toscrape.com/catalogue/category/books/fiction_10/index.html"
}

all_books_data = []

for category_name, category_start_url in category_urls.items():
    print(f"\nCatégorie : {category_name}")

    current_page_url = category_start_url

    while current_page_url:
        response_page = requests.get(current_page_url, headers=request_headers)
        selector_page = Selector(text=response_page.text)

        book_links_on_page = selector_page.css("article.product_pod h3 a::attr(href)").getall()

        for book_relative_link in book_links_on_page:
            book_page_url = requests.compat.urljoin(current_page_url, book_relative_link)

            response_book_page = requests.get(book_page_url, headers=request_headers)
            selector_book_page = Selector(text=response_book_page.text)

            book_title = selector_book_page.css('h1::text').get() or "Titre inconnu"
            book_price = selector_book_page.css('.price_color::text').get() or "£0.00"

            stock_text_parts = selector_book_page.css('.instock.availability::text').getall()
            book_stock_status = ''.join(part.strip() 
            for part in stock_text_parts).strip()

            image_relative_path = selector_book_page.css('div.item.active img::attr(src)').get()
            book_image_url = requests.compat.urljoin(base_url, image_relative_path) if image_relative_path else None

            book_ratings = selector_book_page.css('p.star-rating').attrib.get('class', '')
            book_rating = book_ratings.replace('star-rating', '').strip() if book_ratings else "No rating"

            all_books_data.append({
                "titre": book_title ,
                "prix": book_price ,
                "stock": book_stock_status ,
                "image": book_image_url ,
                "categorie": book_category_name ,
                "url": book_page_url ,
                "rating": book_rating
            })

            print(" -", book_title, ",", book_price, ",", book_stock_status, ",", book_rating)

      
        next_page_link = selector_page.css("li.next a::attr(href)").get()
        if next_page_link:
            current_page_url = requests.compat.urljoin(current_page_url, next_page_link)
        else:
            break

dataframe_books = pd.DataFrame(all_books_data)
dataframe_books.to_csv("books_3cats_nobug.csv", index=False)
dataframe_books.head()



Catégorie : Poetry
 - A Light in the Attic , Â£51.77 , In stock (22 available) , Three
 - The Black Maria , Â£52.15 , In stock (19 available)In stockIn stockIn stockIn stockIn stockIn stock , One
 - Shakespeare's Sonnets , Â£20.66 , In stock (19 available)In stockIn stockIn stockIn stockIn stockIn stock , Four
 - Olio , Â£23.88 , In stock (19 available)In stockIn stockIn stockIn stockIn stockIn stock , One
 - You can't bury them all: Poems , Â£33.63 , In stock (17 available)In stockIn stockIn stockIn stockIn stockIn stock , Two
 - Slow States of Collapse: Poems , Â£57.31 , In stock (17 available)In stockIn stockIn stockIn stockIn stockIn stock , Three
 - Untitled Collection: Sabbath Poems 2014 , Â£14.27 , In stock (16 available)In stockIn stockIn stockIn stockIn stockIn stock , Four
 - Poems That Make Grown Women Cry , Â£14.19 , In stock (15 available)In stockIn stockIn stockIn stockIn stockIn stock , Four
 - Night Sky with Exit Wounds , Â£41.05 , In stock (15 available)In stockIn sto

,titre,prix,stock,image,categorie,url,rating
0,A Light in the Attic,Â£51.77,In stock (22 available),https://books.toscrape.com/media/cache/fe/72/f...,Travel,https://books.toscrape.com/catalogue/a-light-i...,Three
1,The Black Maria,Â£52.15,In stock (19 available)In stockIn stockIn stoc...,https://books.toscrape.com/media/cache/d1/7a/d...,Travel,https://books.toscrape.com/catalogue/the-black...,One
2,Shakespeare's Sonnets,Â£20.66,In stock (19 available)In stockIn stockIn stoc...,https://books.toscrape.com/media/cache/4d/7a/4...,Travel,https://books.toscrape.com/catalogue/shakespea...,Four
3,Olio,Â£23.88,In stock (19 available)In stockIn stockIn stoc...,https://books.toscrape.com/media/cache/b1/0e/b...,Travel,https://books.toscrape.com/catalogue/olio_984/...,One
4,You can't bury them all: Poems,Â£33.63,In stock (17 available)In stockIn stockIn stoc...,https://books.toscrape.com/media/cache/b7/44/b...,Travel,https://books.toscrape.com/catalogue/you-cant-...,Two
